In [44]:
# 분석에 필요한 라이브러리 불러오기
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

df = pd.read_csv('./input/diabetic_data.csv', na_values='?', low_memory=False) 
# 결측치는 ?로 표시되어 있습니다.
df.tail() 

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
101761,443847548,100162476,AfricanAmerican,Male,[70-80),NaN,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),NaN,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),NaN,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),NaN,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO
101765,443867222,175429310,Caucasian,Male,[70-80),NaN,1,1,7,6,...,No,No,No,No,No,No,No,No,No,NO


In [45]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

# Filling missing values
We fill missing values with the mean value for numerical features and the most frequent value for categorical features.

## 데이터 누락된 값 처리?
- weight, payer_code, medical_specialty 는 피쳐 삭제
- race는 최빈값으로 채우기
- diag_1 - 3 는 데이터 삭제

In [46]:
df_null = df.isnull().apply(sum)
df_null[df_null > 0]

race                  2273
weight               98569
payer_code           40256
medical_specialty    49949
diag_1                  21
diag_2                 358
diag_3                1423
dtype: int64

In [47]:
del_columns = ["weight", "payer_code", "medical_specialty","encounter_id","patient_nbr"]
df.drop(del_columns, inplace=True, axis=1)
df['race'] = df['race'].fillna(df['race'].mode()[0])
df.dropna(inplace=True)

In [48]:
# Non-Important Drugs 
del_columns = ['chlorpropamide','acetohexamide', 'troglitazone', 'glyburide-metformin', 'glimepiride-pioglitazone', 'metformin-pioglitazone']
df.drop(del_columns, inplace=True, axis=1)

In [49]:
df_null = df.isnull().apply(sum)
df_null[df_null > 0]

Series([], dtype: int64)

데이터 빈것 없는것을 확인했다.
레이블을 따로 뺀다

In [50]:
df['readmitted'].replace({'NO':0, '>30':1, '<30':1}, inplace = True)
labels = df.pop('readmitted')

In [51]:
# df[['diag_1','diag_2','diag_3']].replace(regex={r'^[VE][0-9]+$':100}, inplace = False) # other diag for 100
df['diag_1'].replace(regex={r'^[VE][0-9]+$':100}, inplace = True)
df['diag_2'].replace(regex={r'^[VE][0-9]+$':100}, inplace = True)
df['diag_3'].replace(regex={r'^[VE][0-9]+$':100}, inplace = True)

In [52]:
df['diag_1'] = df['diag_1'].astype(np.float64)
df['diag_2'] = df['diag_2'].astype(np.float64)
df['diag_3'] = df['diag_3'].astype(np.float64)

In [53]:
diag = ['diag_1','diag_2','diag_3']
for i in diag:
    conditions = [
        (df[i] <= 100 ),
        (df[i] >= 140 ) & (df[i] < 240),
        (df[i] >= 250 ) & (df[i] < 251),
        (df[i] >= 390 ) & (df[i] < 460),
        (df[i] >= 460 ) & (df[i] < 520),
        (df[i] >= 520 ) & (df[i] < 580),
        (df[i] >= 580 ) & (df[i] < 630),
        (df[i] >= 710 ) & (df[i] < 740),
        (df[i] >= 800 ) & (df[i] < 1000),        
        (df[i] == 785),
        (df[i] == 786),
        (df[i] == 787),
        (df[i] == 788)]
    choices = ['other','neoplasms','diabetes','circulatory','respiratory','digestive',
               'genitourinanry','musculoskeletal','injury','circulatory','respiratory','digestive','genitourinanry']
    df[i] = np.select(conditions, choices)
    
df['age'].replace({ '[0-10)':5, '[10-20)':15, 
                 '[20-30)':25, '[30-40)':35, 
                 '[40-50)':45, '[50-60)':55, 
                 '[60-70)':65, '[70-80)':75,
                 '[80-90)':85, '[90-100)':95}, inplace = True)

In [54]:
conditions = [(df['number_diagnoses'] <= 1 ),
    (df['number_diagnoses'] >= 2 ) & (df['number_diagnoses'] < 6),
    (df['number_diagnoses'] >= 6 ) & (df['number_diagnoses'] < 9),
    (df['number_diagnoses'] >= 9 )]
choices = [0, 1, 2, 3]
df['number_diagnoses'] = np.select(conditions, choices)

In [55]:
def num2cat(x):
    return str(x)
obj_list = ['admission_type_id','discharge_disposition_id','admission_source_id']
for i in obj_list:
    df[i] = df[i].apply(num2cat)

# pandas dataframe columns scaling with sklearn


In [56]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

for col in df.dtypes[df.dtypes == 'int64'].index:
    df[[col]] = scaler.fit_transform(df[[col]])

# Getting Dummies from all other categorical features
Apply one hot encoding of categorical features

In [57]:
df.dtypes[df.dtypes == 'object'].index

Index(['race', 'gender', 'admission_type_id', 'discharge_disposition_id',
       'admission_source_id', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'glimepiride',
       'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
       'rosiglitazone', 'acarbose', 'miglitol', 'tolazamide', 'examide',
       'citoglipton', 'insulin', 'glipizide-metformin',
       'metformin-rosiglitazone', 'change', 'diabetesMed'],
      dtype='object')

In [58]:
for col in df.dtypes[df.dtypes == 'object'].index:
    for_dummy = df.pop(col)
    df = pd.concat([df, pd.get_dummies(for_dummy, prefix=col)], axis=1)

In [59]:
df.dtypes

age                               float64
time_in_hospital                  float64
num_lab_procedures                float64
num_procedures                    float64
num_medications                   float64
number_outpatient                 float64
number_emergency                  float64
number_inpatient                  float64
number_diagnoses                  float64
race_AfricanAmerican                uint8
race_Asian                          uint8
race_Caucasian                      uint8
race_Hispanic                       uint8
race_Other                          uint8
gender_Female                       uint8
gender_Male                         uint8
gender_Unknown/Invalid              uint8
admission_type_id_1                 uint8
admission_type_id_2                 uint8
admission_type_id_3                 uint8
admission_type_id_4                 uint8
admission_type_id_5                 uint8
admission_type_id_6                 uint8
admission_type_id_7               

In [60]:
labels.dtypes

dtype('int64')

# KERAS model


In [63]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(df, labels, test_size=0.25)
# print(x_train.shape, y_train.shape)
# print(x_test.shape, y_test.shape)
df.shape

(100244, 165)

In [ ]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=165, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=5000, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, df.values, labels.values, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/50
90218/90218 [==============================] - 1s 10us/step - loss: 0.6896 - acc: 0.5364
Epoch 2/50
90218/90218 [==============================] - 0s 2us/step - loss: 0.6845 - acc: 0.5449
Epoch 3/50
90218/90218 [==============================] - 0s 2us/step - loss: 0.6790 - acc: 0.5677
Epoch 4/50
90218/90218 [==============================] - 0s 3us/step - loss: 0.6740 - acc: 0.5803
Epoch 5/50
90218/90218 [==============================] - 0s 2us/step - loss: 0.6695 - acc: 0.5880
Epoch 6/50
90218/90218 [==============================] - 0s 2us/step - loss: 0.6655 - acc: 0.5940
Epoch 7/50
90218/90218 [==============================] - 0s 2us/step - loss: 0.6627 - acc: 0.5974
Epoch 8/50
90218/90218 [==============================] - 0s 2us/step - loss: 0.6601 - acc: 0.6010
Epoch 9/50
90218/90218 [==============================] - 0s 2us/step - loss: 0.6573 - acc: 0.6042
Epoch 10/50
90218/90218 [==============================] - 0s 2us/step - loss: 0.6554 - acc: 0.6057
Epoch 11

https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/

In [69]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=165, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=200, batch_size=1000, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, df.values, labels.values, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/200
90218/90218 [==============================] - 2s 20us/step - loss: 0.6840 - acc: 0.5509
Epoch 2/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6624 - acc: 0.5966
Epoch 3/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6503 - acc: 0.6092
Epoch 4/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6424 - acc: 0.6175
Epoch 5/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6364 - acc: 0.6251
Epoch 6/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6331 - acc: 0.6306
Epoch 7/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6315 - acc: 0.6325
Epoch 8/200
90218/90218 [==============================] - 0s 6us/step - loss: 0.6316 - acc: 0.6310
Epoch 9/200
90218/90218 [==============================] - 0s 6us/step - loss: 0.6293 - acc: 0.6360
Epoch 10/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6276 - acc: 0.636

90218/90218 [==============================] - 1s 6us/step - loss: 0.6099 - acc: 0.6583
Epoch 83/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6100 - acc: 0.6573
Epoch 84/200
90218/90218 [==============================] - 0s 5us/step - loss: 0.6097 - acc: 0.6588
Epoch 85/200
90218/90218 [==============================] - 0s 6us/step - loss: 0.6084 - acc: 0.6598
Epoch 86/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6085 - acc: 0.6592
Epoch 87/200
90218/90218 [==============================] - 0s 5us/step - loss: 0.6084 - acc: 0.6601
Epoch 88/200
90218/90218 [==============================] - 0s 6us/step - loss: 0.6082 - acc: 0.6589
Epoch 89/200
90218/90218 [==============================] - 0s 5us/step - loss: 0.6078 - acc: 0.6603
Epoch 90/200
90218/90218 [==============================] - 0s 5us/step - loss: 0.6080 - acc: 0.6594
Epoch 91/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6081 - acc: 0.6592
Epo

90218/90218 [==============================] - 1s 6us/step - loss: 0.6010 - acc: 0.6660
Epoch 163/200
90218/90218 [==============================] - 0s 5us/step - loss: 0.6013 - acc: 0.6664
Epoch 164/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6013 - acc: 0.6667
Epoch 165/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6012 - acc: 0.6660
Epoch 166/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6011 - acc: 0.6661
Epoch 167/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6005 - acc: 0.6669
Epoch 168/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.5999 - acc: 0.6675
Epoch 169/200
90218/90218 [==============================] - 0s 5us/step - loss: 0.6004 - acc: 0.6674
Epoch 170/200
90218/90218 [==============================] - 1s 6us/step - loss: 0.6011 - acc: 0.6658
Epoch 171/200
90218/90218 [==============================] - 0s 5us/step - loss: 0.5997 - acc: 0

90219/90219 [==============================] - 0s 5us/step - loss: 0.6143 - acc: 0.6536
Epoch 43/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6137 - acc: 0.6535
Epoch 44/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6138 - acc: 0.6544
Epoch 45/200
90219/90219 [==============================] - 0s 5us/step - loss: 0.6127 - acc: 0.6553
Epoch 46/200
90219/90219 [==============================] - 0s 5us/step - loss: 0.6140 - acc: 0.6542
Epoch 47/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6138 - acc: 0.6539
Epoch 48/200
90219/90219 [==============================] - 0s 6us/step - loss: 0.6131 - acc: 0.6548
Epoch 49/200
90219/90219 [==============================] - 0s 5us/step - loss: 0.6121 - acc: 0.6564
Epoch 50/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6124 - acc: 0.6563
Epoch 51/200
90219/90219 [==============================] - 0s 5us/step - loss: 0.6123 - acc: 0.6563
Epo

90219/90219 [==============================] - 0s 5us/step - loss: 0.6039 - acc: 0.6631
Epoch 124/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6035 - acc: 0.6640
Epoch 125/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6029 - acc: 0.6640
Epoch 126/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6033 - acc: 0.6648
Epoch 127/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6039 - acc: 0.6637
Epoch 128/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6037 - acc: 0.6628
Epoch 129/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6032 - acc: 0.6633
Epoch 130/200
90219/90219 [==============================] - 0s 5us/step - loss: 0.6033 - acc: 0.6632
Epoch 131/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6030 - acc: 0.6638
Epoch 132/200
90219/90219 [==============================] - 1s 6us/step - loss: 0.6027 - acc: 0

...

90220/90220 [==============================] - 1s 6us/step - loss: 0.6365 - acc: 0.6273
Epoch 6/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6326 - acc: 0.6326
Epoch 7/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6310 - acc: 0.6342
Epoch 8/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6292 - acc: 0.6351
Epoch 9/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6302 - acc: 0.6337
Epoch 10/200
90220/90220 [==============================] - 0s 6us/step - loss: 0.6290 - acc: 0.6349
Epoch 11/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6263 - acc: 0.6396
Epoch 12/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6258 - acc: 0.6398
Epoch 13/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6253 - acc: 0.6412
Epoch 14/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6241 - acc: 0.6429
Epoch 1

90220/90220 [==============================] - 1s 6us/step - loss: 0.6084 - acc: 0.6601
Epoch 87/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6084 - acc: 0.6603
Epoch 88/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6086 - acc: 0.6581
Epoch 89/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6083 - acc: 0.6599
Epoch 90/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6079 - acc: 0.6588
Epoch 91/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6081 - acc: 0.6595
Epoch 92/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6086 - acc: 0.6590
Epoch 93/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6074 - acc: 0.6609
Epoch 94/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6088 - acc: 0.6594
Epoch 95/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6072 - acc: 0.6597
Epo

90220/90220 [==============================] - 1s 6us/step - loss: 0.6005 - acc: 0.6675
Epoch 167/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6009 - acc: 0.6674
Epoch 168/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6012 - acc: 0.6661
Epoch 169/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6008 - acc: 0.6671
Epoch 170/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6008 - acc: 0.6664
Epoch 171/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6010 - acc: 0.6664
Epoch 172/200
90220/90220 [==============================] - 0s 6us/step - loss: 0.6010 - acc: 0.6671
Epoch 173/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6006 - acc: 0.6672
Epoch 174/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6005 - acc: 0.6662
Epoch 175/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6010 - acc: 0

90220/90220 [==============================] - 0s 5us/step - loss: 0.6140 - acc: 0.6530
Epoch 47/200
90220/90220 [==============================] - 0s 6us/step - loss: 0.6130 - acc: 0.6537
Epoch 48/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6128 - acc: 0.6548
Epoch 49/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6126 - acc: 0.6553
Epoch 50/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6124 - acc: 0.6542
Epoch 51/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6123 - acc: 0.6548
Epoch 52/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6118 - acc: 0.6559
Epoch 53/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6127 - acc: 0.6556
Epoch 54/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6123 - acc: 0.6551
Epoch 55/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6119 - acc: 0.6563
Epo

Epoch 127/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6033 - acc: 0.6636
Epoch 128/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6036 - acc: 0.6647
Epoch 129/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6039 - acc: 0.6648
Epoch 130/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6030 - acc: 0.6658
Epoch 131/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6035 - acc: 0.6635
Epoch 132/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6034 - acc: 0.6643
Epoch 133/200
90220/90220 [==============================] - 1s 6us/step - loss: 0.6028 - acc: 0.6658
Epoch 134/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6027 - acc: 0.6659
Epoch 135/200
90220/90220 [==============================] - 0s 5us/step - loss: 0.6028 - acc: 0.6665
Epoch 136/200
90220/90220 [==============================] - 0s 5us/step - loss: 0

In [70]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=165, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=1000, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, df.values, labels.values, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/1000
90218/90218 [==============================] - 2s 22us/step - loss: 0.6819 - acc: 0.5537
Epoch 2/1000
90218/90218 [==============================] - 1s 7us/step - loss: 0.6591 - acc: 0.6002
Epoch 3/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6476 - acc: 0.6129
Epoch 4/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6399 - acc: 0.6202
Epoch 5/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6355 - acc: 0.6260
Epoch 6/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6306 - acc: 0.6337
Epoch 7/1000
90218/90218 [==============================] - 0s 5us/step - loss: 0.6296 - acc: 0.6335
Epoch 8/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6274 - acc: 0.6369
Epoch 9/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6272 - acc: 0.6380
Epoch 10/1000
90218/90218 [==============================] - 0s 5us/step - loss: 0.6251 - 

90218/90218 [==============================] - 1s 6us/step - loss: 0.6102 - acc: 0.6567
Epoch 82/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6098 - acc: 0.6565
Epoch 83/1000
90218/90218 [==============================] - 0s 6us/step - loss: 0.6094 - acc: 0.6566
Epoch 84/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6093 - acc: 0.6565
Epoch 85/1000
90218/90218 [==============================] - 0s 5us/step - loss: 0.6091 - acc: 0.6582
Epoch 86/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6100 - acc: 0.6565
Epoch 87/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6100 - acc: 0.6575
Epoch 88/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6092 - acc: 0.6571
Epoch 89/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6117 - acc: 0.6545
Epoch 90/1000
90218/90218 [==============================] - 0s 5us/step - loss: 0.6086 - acc: 0

90218/90218 [==============================] - 0s 5us/step - loss: 0.6029 - acc: 0.6639
Epoch 161/1000
90218/90218 [==============================] - 0s 5us/step - loss: 0.6023 - acc: 0.6641
Epoch 162/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6032 - acc: 0.6633
Epoch 163/1000
90218/90218 [==============================] - 0s 5us/step - loss: 0.6030 - acc: 0.6637
Epoch 164/1000
90218/90218 [==============================] - 0s 6us/step - loss: 0.6030 - acc: 0.6646
Epoch 165/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6028 - acc: 0.6647
Epoch 166/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6032 - acc: 0.6633
Epoch 167/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6036 - acc: 0.6632
Epoch 168/1000
90218/90218 [==============================] - 0s 6us/step - loss: 0.6026 - acc: 0.6643
Epoch 169/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.6035

90218/90218 [==============================] - 1s 6us/step - loss: 0.5998 - acc: 0.6665
Epoch 240/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5988 - acc: 0.6672
Epoch 241/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5991 - acc: 0.6682
Epoch 242/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5993 - acc: 0.6670
Epoch 243/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5991 - acc: 0.6670
Epoch 244/1000
90218/90218 [==============================] - 0s 5us/step - loss: 0.6001 - acc: 0.6663
Epoch 245/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5994 - acc: 0.6666
Epoch 246/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5986 - acc: 0.6671
Epoch 247/1000
90218/90218 [==============================] - 0s 6us/step - loss: 0.5989 - acc: 0.6672
Epoch 248/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5991

90218/90218 [==============================] - 1s 6us/step - loss: 0.5968 - acc: 0.6688
Epoch 319/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5972 - acc: 0.6695
Epoch 320/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5968 - acc: 0.6699
Epoch 321/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5972 - acc: 0.6680
Epoch 322/1000
90218/90218 [==============================] - 0s 5us/step - loss: 0.5961 - acc: 0.6694
Epoch 323/1000
90218/90218 [==============================] - 0s 6us/step - loss: 0.5980 - acc: 0.6688
Epoch 324/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5970 - acc: 0.6690
Epoch 325/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5973 - acc: 0.6694
Epoch 326/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5969 - acc: 0.6683
Epoch 327/1000
90218/90218 [==============================] - 1s 6us/step - loss: 0.5968

...

90220/90220 [==============================] - 1s 6us/step - loss: 0.5846 - acc: 0.6809
Epoch 650/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5842 - acc: 0.6827
Epoch 651/1000
90220/90220 [==============================] - 0s 5us/step - loss: 0.5842 - acc: 0.6823
Epoch 652/1000
90220/90220 [==============================] - 0s 5us/step - loss: 0.5845 - acc: 0.6809
Epoch 653/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5860 - acc: 0.6812
Epoch 654/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5850 - acc: 0.6800
Epoch 655/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5849 - acc: 0.6812
Epoch 656/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5848 - acc: 0.6814
Epoch 657/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5852 - acc: 0.6810
Epoch 658/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5847

90220/90220 [==============================] - 1s 6us/step - loss: 0.5833 - acc: 0.6835
Epoch 729/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5828 - acc: 0.6823
Epoch 730/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5828 - acc: 0.6829
Epoch 731/1000
90220/90220 [==============================] - 0s 6us/step - loss: 0.5833 - acc: 0.6817
Epoch 732/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5834 - acc: 0.6826
Epoch 733/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5834 - acc: 0.6823
Epoch 734/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5829 - acc: 0.6835
Epoch 735/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5825 - acc: 0.6831
Epoch 736/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5830 - acc: 0.6830
Epoch 737/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5832

90220/90220 [==============================] - 0s 5us/step - loss: 0.5819 - acc: 0.6827
Epoch 808/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5817 - acc: 0.6834
Epoch 809/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5817 - acc: 0.6845
Epoch 810/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5821 - acc: 0.6842
Epoch 811/1000
90220/90220 [==============================] - 0s 5us/step - loss: 0.5825 - acc: 0.6828
Epoch 812/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5832 - acc: 0.6825
Epoch 813/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5817 - acc: 0.6844
Epoch 814/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5817 - acc: 0.6836
Epoch 815/1000
90220/90220 [==============================] - 0s 6us/step - loss: 0.5816 - acc: 0.6836
Epoch 816/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5814

90220/90220 [==============================] - 1s 6us/step - loss: 0.5801 - acc: 0.6847
Epoch 887/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5809 - acc: 0.6835
Epoch 888/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5804 - acc: 0.6844
Epoch 889/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5812 - acc: 0.6834
Epoch 890/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5813 - acc: 0.6838
Epoch 891/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5811 - acc: 0.6841
Epoch 892/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5806 - acc: 0.6845
Epoch 893/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5807 - acc: 0.6844
Epoch 894/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5806 - acc: 0.6841
Epoch 895/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5814

90220/90220 [==============================] - 0s 5us/step - loss: 0.5797 - acc: 0.6850
Epoch 966/1000
90220/90220 [==============================] - 1s 7us/step - loss: 0.5793 - acc: 0.6849
Epoch 967/1000
90220/90220 [==============================] - 0s 5us/step - loss: 0.5801 - acc: 0.6824
Epoch 968/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5803 - acc: 0.6832
Epoch 969/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5793 - acc: 0.6856
Epoch 970/1000
90220/90220 [==============================] - 0s 5us/step - loss: 0.5806 - acc: 0.6839
Epoch 971/1000
90220/90220 [==============================] - 0s 5us/step - loss: 0.5798 - acc: 0.6840
Epoch 972/1000
90220/90220 [==============================] - 1s 6us/step - loss: 0.5796 - acc: 0.6839
Epoch 973/1000
90220/90220 [==============================] - 0s 5us/step - loss: 0.5801 - acc: 0.6840
Epoch 974/1000
90220/90220 [==============================] - 0s 5us/step - loss: 0.5794

# 7. Evaluate The Model with k-Fold Cross Validation
We can now evaluate the neural network model on our training data.

The scikit-learn has excellent capability to evaluate models using a suite of techniques. The gold standard for evaluating machine learning models is k-fold cross validation.

First we can define the model evaluation procedure. Here, we set the number of folds to be 10 (an excellent default) and to shuffle the data before partitioning it.

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

Now we can evaluate our model (estimator) on our dataset (X and dummy_y) using a 10-fold cross-validation procedure (kfold).

Evaluating the model only takes approximately 10 seconds and returns an object that describes the evaluation of the 10 constructed models for each of the splits of the dataset.


results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

The results are summarized as both the mean and standard deviation of the model accuracy on the dataset. This is a reasonable estimation of the performance of the model on unseen data. It is also within the realm of known top results for this problem.


Accuracy: 97.33% (4.42%)


In [ ]:

# 쓸모 없어 보이는 데이터 삭제
# del_columns = ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "weight",
#                "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone",
#                "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", 
#                "examide", "citoglipton", "insulin", "glyburide-metformin", "glipizide-metformin", 
#                "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone","number_emergency", "max_glu_serum","number_inpatient"] # 삭제할 데이터
# del_columns = ["weight", "payer_code", "medical_specialty"]
# df.drop(del_columns, inplace=True, axis=1)

In [ ]:
df.head()

# Model selection and tuning
This is the core of this post. We will use H2O AutoML for model selection and tuning.

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

In [ ]:
df['readmit'] = labels
df.tail()

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(df, labels, test_size=0.25)
# print(x_train.shape, y_train.shape)
# print(x_test.shape, y_test.shape)

We load the train and test data on H2O and select the training features and target feature.

In [ ]:
htrain= h2o.H2OFrame(df)
# htest = h2o.H2OFrame(test)

In [ ]:
x = htrain.columns
y ='readmit'
x.remove(y)
# This line is added in the case of classification
htrain[y] = htrain[y].asfactor()

For the AutoML function, we just specify how long we want to train for and we’re set. For this example, we will train for 120 seconds.

In [ ]:
aml = H2OAutoML(max_runtime_secs = 1200)
aml.train(x=x, y =y, training_frame=htrain)
lb = aml.leaderboard
print (lb)
print('Generate predictions…')


In 1200 seconds, AutoML trained 14 models. Some of these models are Gradient Boosting, Extra trees, Random Forest and Deep learning models. Also, it performed stacking of these models to get better AUC score.

This very powerful and saves a lot of time when first deciding on the model choice and parameters and can put you on the right direction.


In [ ]:

# The leader model is stored here
aml.leader

# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

# or:
preds = aml.leader.predict(test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

In [ ]:
y_pred = rf.predict(x_test)
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [ ]:
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
train_results = []
test_results = []

for estimator in n_estimators:
    rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
    rf.fit(x_train, y_train)
    train_pred = rf.predict(x_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = rf.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)

In [ ]:
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(n_estimators, train_results, 'b', label="Train AUC")
line2, = plt.plot(n_estimators, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("n_estimators")
plt.show()

In [ ]:
max_depths = np.linspace(1, 100, 100, endpoint=True)
train_results = []
test_results = []
for max_depth in max_depths:
    rf = RandomForestClassifier(max_depth=max_depth, n_jobs=-1)
    rf.fit(x_train, y_train)
    train_pred = rf.predict(x_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = rf.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)
    

In [ ]:
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(max_depths, train_results, 'b', label="Train AUC")
line2, = plt.plot(max_depths, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("Tree depth")
plt.show()

We see that our model overfits for large depth values. The tree perfectly predicts all of the train data, however, it fails to generalize the findings for new data
# min_samples_split
min_samples_split represents the minimum number of samples required to split an internal node. This can vary between considering at least one sample at each node to considering all of the samples at each node. When we increase this parameter, the tree becomes more constrained as it has to consider more samples at each node. Here we will vary the parameter from 10% to 100% of the samples

In [ ]:
min_samples_splits = np.linspace(0.1, 1.0, 10, endpoint=True)
train_results = []
test_results = []
for min_samples_split in min_samples_splits:
    rf = RandomForestClassifier(min_samples_split=min_samples_split, max_depth = 15, n_estimators = 25)
    rf.fit(x_train, y_train)
    train_pred = rf.predict(x_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = rf.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)    

In [ ]:
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(min_samples_splits, train_results, 'b', label='Train AUC')
line2, = plt.plot(min_samples_splits, test_results, 'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('min samples split')
plt.show()

We can clearly see that when we require all of the samples at each node, the model cannot learn enough about the data. This is an underfitting case.

# min_samples_leaf
min_samples_leaf is The minimum number of samples required to be at a leaf node. This parameter is similar to min_samples_splits, however, this describe the minimum number of samples of samples at the leafs, the base of the tree.

In [ ]:
min_samples_leafs = np.linspace(0.1, 0.5, 5, endpoint=True)
train_results = []
test_results = []
for min_samples_leaf in min_samples_leafs:
    rf = RandomForestClassifier(min_samples_leaf=min_samples_leaf, min_samples_split=0.1, max_depth = 15, n_estimators = 25)
    rf.fit(x_train, y_train)
    train_pred = rf.predict(x_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = rf.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)
    

In [ ]:
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(min_samples_leafs, train_results, 'b', label='Train AUC')
line2, = plt.plot(min_samples_leafs, test_results, 'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('min samples leaf')
plt.show()

Same conclusion as to previous parameter. Increasing this value can cause underfitting.
# max_features
max_features represents the number of features to consider when looking for the best split.

In [ ]:
max_features = list(range(1,train.shape[1]))
train_results = []
test_results = []
for max_feature in max_features:
    rf = RandomForestClassifier(max_features=max_feature,
                               min_samples_leaf=0.1, min_samples_split=0.1, max_depth = 15, n_estimators = 25)
    rf.fit(x_train, y_train)
    train_pred = rf.predict(x_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = rf.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)

In [ ]:
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(max_features, train_results, 'b', label='Train AUC')
line2, = plt.plot(max_features, test_results, 'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('max features')
plt.show()